In [ ]:
from frictionless import Schema
from rocrate.rocrate import ROCrate

# Create the RO-Crate
crate = ROCrate()

# Create a schema for the RO-Crate metadata
rocrate_schema = Schema.from_descriptor({
    "fields": [
        {"name": "@context", "type": "string"},
        {"name": "@graph", "type": "array"}
    ]
})

rocrate_schema.to_json("rocrate_schema.json")

# Add the rocrate schema file to the crate
schema_file = crate.add_file("rocrate_schema.json", properties={
    "@type": "File",
    "name": "Rocrate Schema",
    "encodingFormat": "application/json"
})

In [48]:
from rocrate.rocrate import ROCrate
from frictionless import Schema, Resource

with Resource("data/free_energy_1a.csv") as resource:
    print(f"Header: {resource.header}")

# Add metadata to the root dataset
root_dataset = crate.root_dataset
root_dataset["name"] = "Example RO-Crate"
root_dataset["description"] = "This RO-Crate includes a Frictionless schema."
root_dataset["license"] = "https://spdx.org/licenses/CC0-1.0"

yaml = crate.add_file("data/pfhub.yaml", properties={
    "name": "PFHub meta data file",
    "encodingFormat": "text/yaml"
})
csv = crate.add_file("data/free_energy_1a.csv", properties={
    "name": "Free Energy",
    "encodingFormat": "text/csv",
    "column_header": resource.header
})

# # Create the Frictionless schema
# schema = Schema.from_descriptor({
#     "fields": [
#         {"name": "x", "type": "number", "constraints": {"required": True}},
#         {"name": "y", "type": "number", "constraints": {"required": True}},
#         {"name": "data_set", "type": "string", "constraints": {"required": True}}
#     ]
# })
# schema.to_json("schema.json")

# # Add the schema file to the crate
# schema_file = crate.add_file("schema.json", properties={
#     "@type": "File",
#     "name": "Schema",
#     "encodingFormat": "application/json"
# })

# Save the crate
crate.write("crate")

Header: ['x', 'y', 'data_set']


In [50]:
from frictionless import Resource, validate

# Validate the ro-crate-metadata.json file
resource = Resource("crate/ro-crate-metadata.json", schema="rocrate_schema.json")
report = resource.validate()

# Print the validation report
print(report)

{'valid': True,
 'stats': {'tasks': 1, 'errors': 0, 'warnings': 0, 'seconds': 0.003},
 'warnings': [],
 'errors': [],
 'tasks': [{'name': 'ro-crate-metadata',
            'type': 'json',
            'valid': True,
            'place': 'crate/ro-crate-metadata.json',
            'labels': [],
            'stats': {'errors': 0, 'warnings': 0, 'seconds': 0.003},
            'warnings': [],
            'errors': []}]}


In [52]:
import json

# Load the ro-crate-metadata.json file
with open("crate/ro-crate-metadata.json") as f:
    metadata = json.load(f)

# Check for required properties in the root dataset
root_dataset = next(item for item in metadata["@graph"] if item["@id"] == "./")
required_properties = ["name", "description", "license"]

missing_properties = [prop for prop in required_properties if prop not in root_dataset]
if missing_properties:
    print(f"Missing properties in root dataset: {missing_properties}")
else:
    print("All required properties are present in the root dataset.")

All required properties are present in the root dataset.
